# YouTube Spam Collection v. 1

## Grupo 2
Alberto Atilio Sbrana Junior
<br>
Luiz Barreto Pedro de Alcântara
<br>
Priscila Portela Costa

# Problema

Trata-se de um problema de classificação binária sobre comentários de vídeos no Youtube.
<br>
Há no total 5 arquivos, separados por artista:
- Psy
- Katy Perry
- LMFAO
- Eminem
- Shakira

# Bibliotecas

In [47]:
import pandas as pd #manipulação de dataframes
import numpy as np #manipulacao matricial
import seaborn as sns #visualização
import matplotlib.pyplot as plt #visualização

# importa algumas biblioteca para plotar dados em 3D        
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Rectangle
from pylab import *

import scipy.optimize  #otimizacao de parametros

import ML_library #biblioteca criada pelo grupo 2

# Dados

Primeiro, vamos carregar cada um dos arquivos em um _dataframe_

In [48]:
df_psy = pd.read_csv('Youtube 01-comments Psy.csv')
df_kat = pd.read_csv('Youtube 04-comments KatyPerry.csv')
df_lma = pd.read_csv('Youtube 07-comments LMFAO.csv')
df_emi = pd.read_csv('Youtube 08-comments Eminem.csv')
df_sha = pd.read_csv('Youtube 09-comments Shakira.csv')

# Tratamento dos dados

Como podemos considerar que um modelo melhora ainda mais quando há uma quantidade maior de dados e consideraremos que todos os comentários são em inglês, vamos unir todos os _datasets_ e acrescentar uma nova coluna que indica em qual vídeo 

Por se tratar de análise de texto, é considerado boa prática realizar as seguintes transformações:
- Todas as palavras para minúsculo
- Remoção de caracteres especiais
- Remoção de stop words
- Igualar textos de contrações de palavras e.g 'plz' = 'please'
- Remoção de emojis
- Strings bizarras e.g \ufeff

Também temos uma variável horária, então precisamos transformar esse dado para que o modelo entenda que o valor hour=23 está próximo do valor hour=0, o que não se reflete quando tratamos essa coluna como se fosse um valor inteiro.
<br>
Uma prática comum é usar transformações trigonométricos para transformação horária.

Primeiramente, vamos unir os _datasets_.

In [49]:
df_psy['origin'] = 'psy'
df_kat['origin'] = 'kat'
df_lma['origin'] = 'lma'
df_emi['origin'] = 'emi'
df_sha['origin'] = 'sha'

In [50]:
df_start = df_psy.append([df_kat, df_lma, df_emi, df_sha]).reset_index(drop=True)

In [51]:
#criação de novas colunas: ano, data e has_date (booleana, para o caso do dataset do eminem)
df_start['DATE'] = pd.to_datetime(df_start['DATE'],infer_datetime_format=True)
df_start['comment_year'] = df_start['DATE'].dt.year
df_start['comment_hour'] = df_start['DATE'].dt.hour
df_start['has_date'] = np.where(df_start['comment_year'] == 1969, False, True)

df_start['comment_len'] = df_start['CONTENT'].str.len()

In [52]:
print('Tamanho do dataset unido: {} registros'.format(len(df_start)))

Tamanho do dataset unido: 1956 registros


In [53]:
df_start.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS,origin,comment_year,comment_hour,has_date,comment_len
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07 06:20:48,"Huh, anyway check out this you[tube] channel: ...",1,psy,2013.0,6.0,True,56
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07 12:37:15,Hey guys check out my new channel and our firs...,1,psy,2013.0,12.0,True,166
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08 17:34:21,just for test I have to say murdev.com,1,psy,2013.0,17.0,True,38
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09 08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1,psy,2013.0,8.0,True,48
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10 16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1,psy,2013.0,16.0,True,39


Como já utilizamos a coluna de data, vamos excluí-la do _dataset_ e remover eventuais duplicados.
<br>
Vamos excuir também a coluna COMMENT_ID.

In [54]:
df_start.drop(['DATE', 'COMMENT_ID'], axis=1, inplace=True)
df_start.drop_duplicates(inplace=True)

In [55]:
df_start.head()

,AUTHOR,CONTENT,CLASS,origin,comment_year,comment_hour,has_date,comment_len
0,Julius NM,"Huh, anyway check out this you[tube] channel: ...",1,psy,2013.0,6.0,True,56
1,adam riyati,Hey guys check out my new channel and our firs...,1,psy,2013.0,12.0,True,166
2,Evgeny Murashkin,just for test I have to say murdev.com,1,psy,2013.0,17.0,True,38
3,ElNino Melendez,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1,psy,2013.0,8.0,True,48
4,GsMega,watch?v=vtaRGgvGtWQ Check this out .﻿,1,psy,2013.0,16.0,True,39


Agora que temos o dataset inicial, vamos separar em _features_ (X) e _target_ (y)

In [56]:
df_start.groupby('CLASS').size().reset_index(name='counts')

,CLASS,counts
0,0,946
1,1,981


Podemos ver que a coluna alvo possui uma quantidade de valores balanceados.

In [57]:
#dicionario pra substituicao de contracoes, girias, typos, stemming
word_dict = {
    'pls': 'please',
    'plz': 'please',
    'plizz': 'please',
    'pplease': 'please',
    'pleassssssssssssssss': 'please',
    'dis': 'this',
    'yt': 'youtube',
    'you[tube]': 'youtube',
    'instagraml': 'instagram',
    'facebook-page': 'facebook',
    'wiredo': 'wierdo',
    'vid': 'video',
    'videoeo': 'video',
    'sub': 'subscribe',
    'thx': 'thanks',
    'suscribe': 'subscribe',
    'allot': 'lot',
    'wat': 'what',
    'should.d': 'should',
    'ilove': 'love',
    'likesubscribescribe': 'subscribe',
    'subscribescribe': 'subscribe',
    'anyoutubehing' : 'youtube',
    'itttttttt': 'it',
    'im': 'i',
    'channelthanks': 'channel'
}

#lista de stop workds da biblioteca NLTK
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves',
              'you','you are','you have','you will',"you had",'your','yours','yourself','yourselves',
              'he','him','his','himself','she',"she is",'her','hers','herself','it',"it is",'its',
              'itself','they','them','their','theirs','themselves','what','which','who','whom','this','that',
              "that will",'these','those','am','is','are','was','were','be','been','being','have',
              'has','had','having','do','does','did','doing','a','an','the','and','but','if',
              'or','because','as','until','while','of','at','by','for','with','about','against','between',
              'into','through','during','before','after','above','below','to','from','up','down',
              'in','out','on','off','over','under','again','further','then','once','here','there',
              'when','where','why','how','all','any','both','each','few','more','most','other',
              'some','such','no','nor','not','only','own','same','so','than','too','very','s','t','can',
              'will','just','don',"do not",'should',"should have",'now','d','ll','m','o','re','ve','y','ain',
              'aren',"are not",'couldn',"could not",'didn',"did not",'doesn',"does not",'hadn',"had not",
              'hasn',"has not",'haven',"have not",'isn',"is not",'ma','mightn',"might not",'mustn',"must not",
              'needn',"need not",'shan',"shall not",'shouldn',"should not",'wasn',"was not",'weren',"were not",
              'won',"will not",'wouldn',"would not", 'ill']

In [79]:
def trata_dados(df, treatment_type):
    
    #transformação trigonometrica da coluna hour
    df['sin_time'] = np.sin(2*np.pi*df['comment_hour']/24)
    df['cos_time'] = np.cos(2*np.pi*df['comment_hour']/24)
        
    #criar dummy pra cada origem
    df = pd.get_dummies(df, columns=['origin', 'has_date'])
    
    #novas feature
    df['has_www'] = df['CONTENT'].str.contains(pat = 'www') 
    df['has_http'] = df['CONTENT'].str.contains(pat = 'http')
    df['has_link'] = df['CONTENT'].str.contains(pat = 'watch\?v=')   
    df['has_website'] = (df['has_www'] == True) | (df['has_http'] == True) | (df['has_link'] == True)
    
    #tratamento de texto
    df.drop('AUTHOR', axis=1, inplace=True)
    df['CONTENT'] = df['CONTENT'].str.lower()
    
    column_list = []
    
    for index, row in df.iterrows():
        content = row['CONTENT']
        
        #retirar emojis e caracteres especiais
        regular_characters = "abcdefghijklmnopqrstuvwxyz " 
        content = ''.join(c for c in content if c in regular_characters)
        
        #transfoma texto em lista de palavras:
        content = content.split()
                
        #substituir contracoes de palavras
        for k, v in word_dict.items():
            content = [c.replace(k, v) for c in content]
        
        #remover stop words
        for sw in stop_words:
            content = [c for c in content if sw != c]
        
        column_list.append(content)
        
    column_list = [word for word in column_list if len(word) > 0]
 
    df['CONTENT'] = pd.Series(column_list)
    
    df.dropna(axis=0, subset=['CONTENT'], inplace=True)
    df = df[df['CONTENT'] != '[]']
    
    #transformar cada palavra em uma feature
    columns = []
    for sublist in column_list:
        for item in sublist:
            columns.append(item) 
        
    columns = set(columns)            
    dict_list = []
        
    #valores em cada coluna
    for index, row in df.iterrows():
        content_words = row['CONTENT']
        
        bag_of_words_dict = {}       
        for c in columns:
            bag_of_words_dict[c] = 0
        
        for w in content_words:
            if treatment_type == 'occurrency':
                bag_of_words_dict[w] = 1
            if treatment_type == 'frequency':
                bag_of_words_dict[w] += 1
            if treatment_type == 'tf-idf':         
                bag_of_words_dict[w] +=1
                
        if treatment_type == 'tf-idf': 
            for w in bag_of_words_dict:
                bag_of_words_dict[w] = bag_of_words_dict[w] / len(content_words)
        
        dict_list.append(bag_of_words_dict)        
     
    df = df.assign(bag_words=dict_list)
    bag_words = df['bag_words'].apply(pd.Series)
    
    if treatment_type == 'tf-idf':
        for c in bag_words:
            if bag_words[c].sum() > 0:
                bag_words[c] = np.log(len(df)/bag_words[c].sum())*bag_words[c]
            else:
                bag_words[c] = 0
        
    df = pd.concat([df, bag_words], axis = 1)
    
    #remover coluna content
    df.drop(['CONTENT','bag_words'], axis=1, inplace=True)
    #remover possiveis nulos
    df.dropna(inplace=True)
     
    return df

In [80]:
treatment_types = ['occurrency', 'frequency', 'tf-idf']
treated_df_dict = {}

for t in treatment_types:
    df_treated = trata_dados(df_start, treatment_type=t)
    treated_df_dict[t] = df_treated

Com isso, temos três _datasets_ com diferentes tipos de preprocessamento, todos salvos no dicionário `treated_df_dict`

# Modelos testados

Serão utilizados os algoritmos ensinados em sala de aula:
- K-Nearest Neighbors (K-NN)
- Regressão Logística
- Classificador Naive-Bayes
- Redes Neurais Artificiais
- Máquinas de Vetores de Suporte

Caso seja necessário ou por motivos de visualização, reduziremos a dimensionalidade do nosso _dataset_ utilizando Análise de Componentes Principais

# Avaliação do modelo

## Métrica de avaliação

Por se tratar de um classificador binário, é possível que ocorram 4 tipos de classificação:
- Verdadeiros positivos
- Verdadeiros negativos
- Falsos positivos
- Falsos negativos

Obviamente, queremos o maior número de verdadeiros positivos e verdadeiros negativos possíveis em nosso modelo.
<br>
<br>
Entretanto, no caso de um classificador de SPAM, pode-se escolher como uma boa métrica uma função que não só leve em conta a acurácia do modelo, mas também uma minimização do número de falsos positivos.
<br>
<br>
Por exemplo, as gravadoras podem querer minimizar a quantidade de contários de fãs que possam ser erroneamente classificados como SPAM, ao custo de deixar um ou outro comentário SPAM ser classificado como legítimo.
<br>
<br>
Portanto, utilizaremos o F1-Score, que é uma média harmônica entre o recall (quantos dos comentários SPAM foram corretamente classificados como SPAM) e a precisão (quantos dos comentários foram classificados corretamente)

\begin{equation*}
F_1   = 2 * \frac{precision * recall} {precision + recall}
\end{equation*}

## Benchmark

Idealmente, todo modelo deve ser avaliado em relação a um classificador _naive_ ou de ponto de partida, que pode, no nosso caso, ser descrito pelo comportamento de uma moeda.
<br>
Com isso, queremos que nosso classificador tenha a seguinte característica:

\begin{equation*}
F_1modelo > F_1naive,    \text{para $F_1naive=0.5$}
\end{equation*}

# Separação de dados: _Stratified holdout_

## Dataframes separados

In [72]:
df_occur = treated_df_dict['occurrency']
df_frequ = treated_df_dict['frequency']
df_tfidf = treated_df_dict['tf-idf']

#separacao X e y
X_occur = df_occur.drop('CLASS', axis=1).values
y_occur = df_occur['CLASS'].values

X_frequ = df_frequ.drop('CLASS', axis=1).values
y_frequ = df_frequ['CLASS'].values

X_tfidf = df_tfidf.drop('CLASS', axis=1).values
y_tfidf = df_tfidf['CLASS'].values

## Parâmetros fixados

In [73]:
# semente usada na randomizacao dos dados.
randomSeed = 10 
# define a porcentagem de dados que irao compor o conjunto de treinamento
pTrain = 0.8 

## Separação de valores de treino e teste para cada um dos _dataframes_

In [74]:
import numpy as np

#gera os indices aleatorios que irao definir a ordem dos dados
idx_perm_occur = np.random.RandomState(randomSeed).permutation(range(len(y_occur)))
idx_perm_frequ = np.random.RandomState(randomSeed).permutation(range(len(y_frequ)))
idx_prem_tfidf = np.random.RandomState(randomSeed).permutation(range(len(y_tfidf)))

#ordena os dados de acordo com os indices gerados aleatoriamente
X2_occur, Y2_occur = X_occur[idx_perm_occur, :], y_occur[idx_perm_occur]
X2_frequ, Y2_frequ = X_frequ[idx_perm_frequ, :], y_frequ[idx_perm_frequ]
X2_tfidf, Y2_tfidf = X_tfidf[idx_prem_tfidf, :], y_tfidf[idx_prem_tfidf]

# obtem os indices dos dados da particao de treinamento e da particao de teste
train_index_occur, test_index_occur = ML_library.stratified_holdOut(Y2_occur, pTrain)
train_index_frequ, test_index_frequ = ML_library.stratified_holdOut(Y2_frequ, pTrain)
train_index_tfidf, test_index_tfidf = ML_library.stratified_holdOut(Y2_tfidf, pTrain)

#datasets de treino e teste
#occurency
X_train_occur, X_test_occur = X2_occur[train_index_occur, :], X2_occur[test_index_occur, :];
Y_train_occur, Y_test_occur = Y2_occur[train_index_occur], Y2_occur[test_index_occur];

#frequency
X_train_frequ, X_test_frequ = X2_frequ[train_index_frequ, :], X2_frequ[test_index_frequ, :];
Y_train_frequ, Y_test_frequ = Y2_frequ[train_index_frequ], Y2_frequ[test_index_frequ];

#tf-idf
X_train_tfidf, X_test_tfidf = X2_tfidf[train_index_tfidf, :], X2_tfidf[test_index_tfidf, :];
Y_train_tfidf, Y_test_tfidf = Y2_tfidf[train_index_tfidf], Y2_tfidf[test_index_tfidf];

In [75]:
## Separação de dados para curva de aprendizado
train_index_occur, val_index_occur = ML_library.stratified_holdOut(Y_train_occur, pTrain)
train_index_frequ, val_index_frequ = ML_library.stratified_holdOut(Y_train_frequ, pTrain)
train_index_tfidf, val_index_tfidf = ML_library.stratified_holdOut(Y_train_tfidf, pTrain)
#occurrency
X_train_v_occur, X_val_occur = X_train_occur[train_index_occur, :], X_train_occur[val_index_occur, :]
Y_train_v_occur, Y_val_occur = Y_train_occur[train_index_occur], Y_train_occur[val_index_occur]
#frequency
X_train_v_frequ, X_val_frequ = X_train_frequ[train_index_frequ, :], X_train_frequ[val_index_frequ, :]
Y_train_v_frequ, Y_val_frequ = Y_train_frequ[train_index_frequ], Y_train_frequ[val_index_frequ]
#tf-idf
X_train_v_tfidf, X_val_tfidf = X_train_tfidf[train_index_tfidf, :], X_train_tfidf[val_index_tfidf, :]
Y_train_v_tfidf, Y_val_tfidf = Y_train_tfidf[train_index_tfidf], Y_train_tfidf[val_index_tfidf]

A escolha do modelo será feita utilizando os _datasets_ de treino e validação e, por fim, serão testados no _dataset_ de teste

In [76]:
#occurrency
np.savetxt('X_occurrency_train.csv', X_train_v_occur, delimiter=",")
np.savetxt('X_occurrency_validation.csv', X_val_occur, delimiter=",")
np.savetxt('X_occurrency_test.csv', X_test_occur, delimiter=",") 

np.savetxt('Y_occurrency_train.csv', Y_train_v_occur, delimiter=",") 
np.savetxt('Y_occurrency_validation.csv', Y_val_occur, delimiter=",") 
np.savetxt('Y_occurrency_test.csv', Y_test_occur, delimiter=",") 

#frequency
np.savetxt('X_frequency_train.csv', X_train_v_frequ, delimiter=",")
np.savetxt('X_frequency_validation.csv', X_val_frequ, delimiter=",")
np.savetxt('X_frequency_test.csv', X_test_frequ, delimiter=",") 

np.savetxt('Y_frequency_train.csv', Y_train_v_frequ, delimiter=",") 
np.savetxt('Y_frequency_validation.csv', Y_val_frequ, delimiter=",") 
np.savetxt('Y_frequency_test.csv', Y_test_frequ, delimiter=",") 

#tf-idf
np.savetxt('X_tfidf_train.csv', X_train_v_tfidf, delimiter=",")
np.savetxt('X_tfidf_validation.csv', X_val_tfidf, delimiter=",")
np.savetxt('X_tfidf_test.csv', X_test_tfidf, delimiter=",") 

np.savetxt('Y_tfidf_train.csv', Y_train_v_tfidf, delimiter=",") 
np.savetxt('Y_tfidf_validation.csv', Y_val_tfidf, delimiter=",") 
np.savetxt('Y_tfidf_test.csv', Y_test_tfidf, delimiter=",") 